In [1]:
import random
import numpy as np

# Divide and conquer

## Merge sort

In [2]:
# Average and Worst case performance: O(n log(n))
def mergesort(array):
    if len(array) <= 1:
        return array
    else:
        mid = len(array) // 2
        left = array[:mid]
        right = array[mid:]
        #print(left, right)
        left = mergesort(left)
        #print(left)
        right = mergesort(right)
        #print(right)
        return merge(left, right)
        

def merge(left, right):
    result = []
    while len(left) > 0 and len(right) > 0:
        result.append(left.pop(0) if left[0] <= right[0] else right.pop(0))
    #while len(left) > 0:
    #    result.append(left.pop(0))
    #while len(right) > 0:
    #    result.append(right.pop(0))
    #return result
    return result + left + right

In [3]:
n = 4
randomlist = random.sample(range(0, n), n)
print(randomlist) if len(randomlist) < 20 else print("List too long to print")

[0, 3, 1, 2]


In [4]:
%%time 
mergesort(randomlist)

CPU times: total: 0 ns
Wall time: 0 ns


[0, 1, 2, 3]

#### Comparison with bubble sort

In [5]:
def bubble_sort(arr):
    n = len(arr)
    
    for i in range(n - 1):
        # Last i elements are already in place
        for j in range(n - i - 1):
            # Swap if the element found is greater than the next element
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    
    return arr

In [6]:
%%time 
bubble_sort(randomlist)

CPU times: total: 0 ns
Wall time: 0 ns


[0, 1, 2, 3]

## Strassen Matrix Multpilication

In [7]:
def strassen_multiply(A, B):
    n = A.shape[0]
    
    # Base case: if the matrices are 64x64 or smaller, use regular matrix multiplication
    if n <= 64:
        return np.dot(A, B)
    
    # Splitting the matrices into quadrants
    mid = n // 2
    A11, A12 = A[:mid, :mid], A[:mid, mid:]
    A21, A22 = A[mid:, :mid], A[mid:, mid:]
    B11, B12 = B[:mid, :mid], B[:mid, mid:]
    B21, B22 = B[mid:, :mid], B[mid:, mid:]
    
    # Recursive steps
    P1 = strassen_multiply(A11 + A22, B11 + B22)
    P2 = strassen_multiply(A21 + A22, B11)
    P3 = strassen_multiply(A11, B12 - B22)
    P4 = strassen_multiply(A22, B21 - B11)
    P5 = strassen_multiply(A11 + A12, B22)
    P6 = strassen_multiply(A21 - A11, B11 + B12)
    P7 = strassen_multiply(A12 - A22, B21 + B22)
    
    # Computing the submatrices of the result matrix
    C11 = P1 + P4 - P5 + P7
    C12 = P3 + P5
    C21 = P2 + P4
    C22 = P1 - P2 + P3 + P6
    
    # Combining the submatrices into a single matrix
    C = np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))
    
    return C


In [8]:
rows = 2048
columns = 2048

# Create random matrices
a = np.random.randint(low=1, high=10, size=(rows, columns))
b = np.random.randint(low=1, high=10, size=(rows, columns))

In [9]:
%%time
strassen_multiply(a, b)

CPU times: total: 875 ms
Wall time: 4.83 s


array([[51277, 52877, 51688, ..., 51769, 51708, 51117],
       [50931, 52674, 50761, ..., 51304, 51122, 50447],
       [50734, 51771, 51359, ..., 50997, 51284, 50094],
       ...,
       [50029, 51909, 50483, ..., 50544, 50471, 49893],
       [49553, 51222, 49659, ..., 50176, 49774, 49400],
       [49303, 50761, 49951, ..., 50804, 50400, 49401]])

### Comparison with classical matrix multiplication

In [10]:
%%time
a @ b

CPU times: total: 15.7 s
Wall time: 1min 5s


array([[51277, 52877, 51688, ..., 51769, 51708, 51117],
       [50931, 52674, 50761, ..., 51304, 51122, 50447],
       [50734, 51771, 51359, ..., 50997, 51284, 50094],
       ...,
       [50029, 51909, 50483, ..., 50544, 50471, 49893],
       [49553, 51222, 49659, ..., 50176, 49774, 49400],
       [49303, 50761, 49951, ..., 50804, 50400, 49401]])